In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.applications import InceptionV3, VGG16, InceptionResNetV2, VGG19, ResNet101, ResNet101V2, ResNet152, ResNet152V2, ResNet50, ResNet50V2, Xception
from time import perf_counter 
import os
import cv2
from tensorflow.keras.optimizers import Adam,SGD,RMSprop

In [ ]:
os.environ['KAGGLE_USERNAME']="ridhimamishra1022"
os.environ['KAGGLE_KEY']="43a84c2ffcbdaa7a111c8577df6d155a"

In [ ]:
!kaggle datasets download -d jonathanoheix/face-expression-recognition-dataset

face-expression-recognition-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip face-expression-recognition-dataset.zip

Archive:  face-expression-recognition-dataset.zip
replace images/images/train/angry/0.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
batch_size = 100
img_height =48
img_width = 48

In [ ]:
training_ds = tf.keras.preprocessing.image_dataset_from_directory(
    '/content/images/train',
    seed=42,
    image_size= (img_height, img_width),
    batch_size=batch_size)

Found 28821 files belonging to 7 classes.


In [ ]:
validation_ds =  tf.keras.preprocessing.image_dataset_from_directory(
    '/content/images/validation',
    seed=42,
    image_size= (img_height, img_width),
    batch_size=batch_size)

Found 7066 files belonging to 7 classes.


In [ ]:
vgg19= VGG19(include_top=False, input_shape=(256,256,3), classes=7)

In [ ]:
X=vgg19.output
X=Flatten()(X)
X=Dense(1024, activation='relu')(X)
X=Dropout(0.3)(X)
X=Dense(512, activation='relu')(X)
X=Dropout(0.2)(X)
X=Dense(256, activation='relu')(X)
output=Dense(7, activation='softmax')(X)

In [ ]:
model=Model(inputs=vgg19.input, outputs=output)

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 256, 256, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 256, 256, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 128, 128, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 128, 128, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 128, 128, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 64, 64, 128)       0     

In [ ]:
model.compile(optimizer='adam', loss=tf.keras.losses.sparse_categorical_crossentropy, metrics=['accuracy'])

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

checkpoint = ModelCheckpoint(filepath="./cnn_model.h5", monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

early_stopping = EarlyStopping(monitor='val_loss',
                          min_delta=0,
                          patience=3,
                          verbose=1,
                          restore_best_weights=True
                          )

reduce_learningrate = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.2,
                              patience=3,
                              verbose=1,
                              min_delta=0.0001)

callbacks_list = [early_stopping,checkpoint,reduce_learningrate]

In [ ]:
model.fit(training_ds, validation_data=validation_ds, batch_size=16, epochs=20,callbacks=callbacks_list)


Epoch 1/20
289/289 [==============================] - ETA: 0s - loss: 2.1396 - accuracy: 0.2447
Epoch 1: val_accuracy improved from -inf to 0.25842, saving model to ./cnn_model.h5
289/289 [==============================] - 680s 2s/step - loss: 2.1396 - accuracy: 0.2447 - val_loss: 1.8115 - val_accuracy: 0.2584 - lr: 0.0010
Epoch 2/20
289/289 [==============================] - ETA: 0s - loss: 1.8034 - accuracy: 0.2541
Epoch 2: val_accuracy improved from 0.25842 to 0.26238, saving model to ./cnn_model.h5
289/289 [==============================] - 641s 2s/step - loss: 1.8034 - accuracy: 0.2541 - val_loss: 1.8090 - val_accuracy: 0.2624 - lr: 0.0010
Epoch 3/20
289/289 [==============================] - ETA: 0s - loss: 1.8004 - accuracy: 0.2529
Epoch 3: val_accuracy improved from 0.26238 to 0.27526, saving model to ./cnn_model.h5
289/289 [==============================] - 641s 2s/step - loss: 1.8004 - accuracy: 0.2529 - val_loss: 1.7801 - val_accuracy: 0.2753 - lr: 0.0010
Epoch 4/20
289/289 

In [ ]:
################MobileNetV2##############

In [ ]:
base_model = tf.keras.applications.MobileNetV2(include_top=False,input_shape=(256,256,3), classes=7)

In [ ]:
X=base_model.output
X=Flatten()(X)
X=Dense(1024, activation='relu')(X)
X=Dropout(0.3)(X)
X=Dense(512, activation='relu')(X)
X=Dropout(0.2)(X)
X=Dense(256, activation='relu')(X)
output=Dense(7, activation='softmax')(X)

In [ ]:
model1=Model(inputs=base_model.input, outputs=output)

In [ ]:
model1.compile(optimizer='adam', loss=tf.keras.losses.sparse_categorical_crossentropy, metrics=['accuracy'])

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

checkpoint = ModelCheckpoint(filepath="./mobilenetV2_model.h5", monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

early_stopping = EarlyStopping(monitor='val_loss',
                          min_delta=0,
                          patience=3,
                          verbose=1,
                          restore_best_weights=True
                          )

reduce_learningrate = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.2,
                              patience=3,
                              verbose=1,
                              min_delta=0.0001)

callbacks_list = [early_stopping,checkpoint,reduce_learningrate]

In [ ]:
model1.fit(training_ds, validation_data=validation_ds, batch_size=16, epochs=20,callbacks=callbacks_list)

Epoch 1/20
289/289 [==============================] - ETA: 0s - loss: 1.8606 - accuracy: 0.3794
Epoch 1: val_accuracy improved from -inf to 0.16134, saving model to ./mobilenetV2_model.h5
289/289 [==============================] - 267s 900ms/step - loss: 1.8606 - accuracy: 0.3794 - val_loss: 3.5372 - val_accuracy: 0.1613 - lr: 0.0010
Epoch 2/20
289/289 [==============================] - ETA: 0s - loss: 1.3563 - accuracy: 0.4829
Epoch 2: val_accuracy did not improve from 0.16134
289/289 [==============================] - 256s 883ms/step - loss: 1.3563 - accuracy: 0.4829 - val_loss: 2.1598 - val_accuracy: 0.1128 - lr: 0.0010
Epoch 3/20
289/289 [==============================] - ETA: 0s - loss: 1.2100 - accuracy: 0.5350
Epoch 3: val_accuracy improved from 0.16134 to 0.16190, saving model to ./mobilenetV2_model.h5
289/289 [==============================] - 261s 900ms/step - loss: 1.2100 - accuracy: 0.5350 - val_loss: 1.9903 - val_accuracy: 0.1619 - lr: 0.0010
Epoch 4/20
289/289 [==========